# Identificación de sistemas por medio de la curva de reacción (respuesta al escalón).


En esta práctica vamos a obtener el modelo por curva de reacción y el modelo estático para un sistema térmico. También lo haremos para un motor DC considerando como salida la velocidad. Usaremos para ello las plantas diseñadas en la universidad denominadas, respectivamente,  UNThermal y UNDCMotor.

+ El UNThermal es un minilaboratorio que permite realizar el control de temperatura de un par transistor-sensor mediante la administración precisa de potencia eléctrica controlada por una fuente de corriente analógica. 

+ El UNDCMotor es un minilaboratorio que permite realizar el control de velocidad y ángulo de giro de un motor DC.


## Configuración
---


### Instalación de las librerias unthermal y unmotor

Ejecutar esta celda solo para instalar por primera vez o actualizar la libreria. __Asegúrese de instalar  [Git](https://git-scm.com/download/win/ "Git").__

<div class="alert alert-block alert-danger">
⚠️ Ejecutar en el terminal de visual studio code estos comandos solo  para instalar la libreria por primera vez
</div>





`pip install --ignore-installed  git+https://github.com/nebisman/UNThermal.git@main#subdirectory=code/python_code`

`pip install --ignore-installed  git+https://github.com/nebisman/UNDCMotor.git@main#subdirectory=code/python_code`


Si la instalación reporta algún error de permisos del sistema operativo, use esta opción:


`pip install --ignore-installed --user git+https://github.com/nebisman/UNThermal.git@main#subdirectory=code/python_code`

`pip install --ignore-installed --user git+https://github.com/nebisman/UNDCMotor.git@main#subdirectory=code/python_code`


### Importación de comandos de la libreria unthermal y unmotor

A continuación importamos los comandos básicos que usaremos de la libreria `unthermal` y la libreria `matplotlib` para las figuras.

In [ ]:
%matplotlib widget
import mplcursors
import matplotlib.pyplot as plt
import unthermal as temp
import numpy as np
import unmotor as mot


---

### Definición del sistema en el IoT

En este enlace encuentra toda la ayuda de las funciones  [enlace de la ayuda](https://nebisman.github.io/UNThermal/)

Lo primero que necesitamos es definir el sistema y su conexión al IoT. Esto lo hacemos mediante el  comando

`MotorSystemIoT(plant_number="xxxx", broker_address = "xx.xxx.xxx.xxx")`.

+ El parámetro `plant_number` es el identificador de la planta con que está trabajando. Por ejemplo `plant_number="PC-M-XX"`

+ El parámetro `broker_address` es la dirección IP del broker al cual se conecta la planta. Poner siempre ` broker_address = "192.168.1.100"` 

## Modelo estático y dinámico para el Motor DC
___

A continuación vamos a realizar el trabajo de un modelo estático y dinámico para el Motor DC. Para ello. primero definimos la planta como objeto IoT. Esto lo hacemos a continuación:

In [ ]:
mi_motor = mot.MotorSystemIoT(plant_number = "PC-M-XX", broker_address = "192.168.1.100") 

### Modelo estático del Motor DC



El modelo estático se puede obtener experimentalmente de varias formas. *otra manera
habitual es ajustar un valor constante de entrada y medir la salida en estado estacionario.
Esto da un punto del modelo estático. Este procedimiento se repite para cubrir el
rango completo de entradas.*

+ La  función `get_static_model(m)` nos permite obtener el modelo estático del UNDCmotor 

+ La función `plt.savefig(.)` permite guardar la figura en en cualquier formato.

In [ ]:
ue, ye = mot.get_static_model(mi_motor) 
plt.savefig("static_model.pdf", format="pdf", bbox_inches="tight") # la figura se guarda en el archivo que ud desee

#### Preguntas orientadoras sobre el modelo estático

Discuta inmediatamente estas preguntas con su compañero e incluya una respuesta breve en su bitácora.

1.  ¿Cuál es la zona muerta del motor?
2.  ¿Hasta que velocidad máxima se puede controlar el motor?
3.  ¿Se puede controlar una velocidad de $700^o/s$ si ocurre una perturbación por carga del motor?
4.  ¿Se puede controlar una velocidad de $10^o/s$? 

### Modelo dinámico por respuesta al escalón 
---

Un motor DC puede ser descrito con muy buena aproximación por un  [modelo lineal de primer orden](https://github.com/nebisman/UNDCMotor/blob/main/docs/modelo_matematico_unmotor.pdf), si consideramos que la entrada es tensión y la salida es la velocidad angular.

Así, el UNDCMotor puede ser descrito por la siguiente función de transferencia:

$$G(s)= \frac{\alpha}{\tau\, s + 1} e^{-sL}$$

*Note que el retardo fundamentalmente aparece por el tiempo de muestreo del sistema digital que procesa la información.*

La función `get_fomodel_step(system, yop = desired_op)`  *(get first order model by using step response)* permite obtener la respuesta al escalón y el modelo matemático del UNDCMotor [Aqui está la ayuda](https://nebisman.github.io/UNThermal/get_fomodel_step.html#)  

+ El parámetro `system` es la instancia del UNDCMotor creada con la función  `MotorSystemIoT(.)`

+ El parámetro `yop = desired_op` permite obtener la respuesta al escalón con un cambio de tensión que centra, aproximadamente, los valores final e inicial del escalón en el punto de operación `yop`.

+  `alpha, tau` y `L` retornan los parámetros del motor.

En el ejemplo siguiente obtenemos la respuesta al escalón y el modelo para un punto de operación `yop=150` . 


In [ ]:
# Ejemplo de uso
alfa, tau, L = mot.get_fomodel_step(mot, yop = 150)
plt.savefig("first_order_model_yop_150.pdf", format="pdf", bbox_inches="tight") # para salvar la figura

### Trabajo experimental con el motor

Encuentre los modelos dinámicos por respuesta al escalón para los siguientes puntos de operación:

+ $y_{OP}=150^o/s$
+ $y_{OP}=200^o/s$
+ $y_{OP}=300^o/s$
+ $y_{OP}=400^o/s$
+ $y_{OP}=500^o/s$
+ $y_{OP}=600^o/s$



#### Preguntas orientadoras 
Realice una tabla con los valores de $\alpha$ y $\tau$ para los diferentes puntos de operación y responda junto con su compañero inmediatamente estas preguntas en su bitácora. 


+ Describa en sus palabras como cambia la constante de tiempo $\tau$ al incrementar la velocidad.
+ Describa en sus parlabras como cambia la constante de ganancia $\alpha$ al incrementar la velocidad.


#### Elaboración


+ Suponga que vamos a controlar el motor en velocidades cercanas a $300^o/s$. Realice los experimentos que sean necesarios para caracterizar la incertidumbre absoluta de los parámetros $\alpha$ y $\tau$ en esta velocidad. Obtenga la incertidumbre absoluta.

## Modelos estático y dinámico del sistema térmico
---

A continuación vamos a obtener el modelo dinámico por respuesta al escalón del sistema térmico. Si el tiempo alcanza, obtendremos el modelo estático. Primero definimos el sistema térmico en el IoT, como sigue:

In [ ]:
sistema_termico = temp.ThermalSystemIoT(plant_number = "PC-T-XX" , broker_address = "192.168.1.100")



### Modelo dinámico por respuesta al escalón

Cuando se realiza la linealización del modelo del UNThermal se obtiene un  modelo lineal de primer orden, tal y como puede revisar en este [documento](https://github.com/nebisman/UNThermal/blob/main/docs/modelo_matematico_unthermal.pdf).

Así, nuestro sistema térmico, dentro de una región cercana al punto de operación, puede ser descrito por la siguiente función de transferencia:

$$G(s)= \frac{\alpha}{\tau\, s + 1} \, e^{-L\,s}$$

Para identificar el sistema térmico con base en la respuesta al escalón usamos la función `get_fomodel_step(system, yop = desired_op)`  *(get first order model by using step response)*. Lo hacemos en el punto de operación `yop=50`. [Aqui está la ayuda](https://nebisman.github.io/UNThermal/get_fomodel_step.html#) 


In [ ]:
# inicialmente hacemos el experimento de respuesta al escalón para 50 grados
t, u, y= temp.get_fomodel_step(sistema_termico, yop=50, t1=600, amplitude=10)

#### Trabajo experimental con el sistema térmico

+ Obtenga el modelo en el punto de operación de  $70^o$.

#### Preguntas orientadoras sobre los modelos dinámicos

+ Realice una tabla con los valores de $\alpha$ y $\tau$ para los dos puntos de operación. Con base en esta tabla, compare la variación de parámetros en relación con la que ocurre en el motor. Inclúyalo inmediatamente en su bitácora.  

### Modelo estático del sistema térmico 



Es natural comenzar investigando el comportamiento estacionario de un sistema bajo
control. Esto se hace mediante una curva que muestra el valor en estado estacionario de la
salida (en este caso, la temperatura del par transistor-sensor) para diferentes valores de la variable de entrada (en este caso la potencia aplicada). A esta curva se le denomina Modelo Estático del Sistema.


El modelo estático se puede obtener experimentalmente de varias formas. *Una manera
es que el sistema esté en lazo cerrado con un controlador preajustado e ir ajustando diferentes valores de la referencia. Cuando la salida (en este caso la temperatura) alcanza el valor estacionario de referencia, se registra la señal de control que fue necesaria. Esto da un punto del modelo estático. Este procedimiento se repite para cubrir el rango completo de entradas.*



+ La  función `get_static_model(system)` nos permite obtener el modelo estático del sistema térmico mediante la función  `ThermalSystemIoT(.)`

#### Experimento
Obtenga el modelo estático del sistema térmico mediante las siguientes instrucciones:

In [ ]:
ue, ye = temp.get_static_model(sistema_termico) 
plt.savefig("static_model.pdf", format="pdf", bbox_inches="tight") # la figura se guarda en el archivo que ud desee

#### Pregunta orientadora

+ Discuta la consistencia del modelo estático del sistema térmico, con los dos modelos dinámicos que obtuvo. Examine si los valores de estado estacionario de los modelos dinámicos son correspondientes. Incluyalo inmediatamente en su bitácora. 

